# MLflow GenAI Model Evaluation

This notebook demonstrates:
1. Logging the LangGraph RAG agent model with MLflow
2. Creating evaluation datasets
3. Running mlflow.genai.evaluate with multiple scorers
4. Analyzing evaluation results

**Prerequisites:** Run langgraph_agent.ipynb first to create the agent code


## 1. Setup and Imports


In [ ]:
%pip install langgraph langchain databricks_agents databricks_langchain databricks-vectorsearch "mlflow>=3.6" pandas matplotlib litellm
%restart_python


In [ ]:
import mlflow
import pandas as pd
from mlflow.genai import evaluate

# MLflow 3.6+ Evaluation Dataset and Scorer Registration imports
import mlflow.genai.datasets
from mlflow.genai.datasets import create_dataset, get_dataset
from mlflow.genai.scorers import (
    Safety, RelevanceToQuery,
    ScorerSamplingConfig, scorer, list_scorers, get_scorer
)
from mlflow.genai.judges import make_judge

print(f"MLflow version: {mlflow.__version__}")
assert mlflow.__version__ >= "3.6.0", "MLflow 3.6+ required for dataset/scorer registration"


## 2. Configuration


In [ ]:
# Unity Catalog configuration
CATALOG_NAME = "brian_ml_dev"
SCHEMA_NAME = "eval_testing"
UC_SCHEMA = f"{CATALOG_NAME}.{SCHEMA_NAME}"  # For dataset/table references

# MLflow configuration
EXPERIMENT_NAME = "/Users/brian.law@databricks.com/langgraph_rag_agent"
EVAL_EXPERIMENT_NAME = "/Users/brian.law@databricks.com/langgraph_rag_evaluation"

# Evaluation Dataset configuration (stored in Unity Catalog)
EVAL_DATASET_NAME = "rag_agent_eval_dataset"
EVAL_DATASET_UC_TABLE = f"{UC_SCHEMA}.{EVAL_DATASET_NAME}"

# Vector Search configuration (from pdf_to_vector_search notebook)
VECTOR_INDEX_NAME = "annual_report_index"
VECTOR_SEARCH_ENDPOINT = "one-env-shared-endpoint-13"

# LLM configuration
LLM_ENDPOINT = "databricks-gpt-oss-120b"
TOP_K_RESULTS = 3

# Judge LLM for evaluation
JUDGE_LLM = "databricks:/databricks-gpt-oss-120b"

# Production monitoring configuration
PRODUCTION_SAMPLE_RATE = 0.5  # Sample 50% of traces for scorer monitoring


## 3. Log Model with MLflow Model As Code


In [ ]:
# Set experiment and log model using Model As Code
mlflow.set_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name="langgraph_rag_agent") as run:
    
    # Log model using MLflow LangChain with Model As Code
    # Just reference the notebook path - no need to instantiate the class!
    full_index_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{VECTOR_INDEX_NAME}"
    
    mlflow.langchain.log_model(
        lc_model="langgraph_agent",  # Reference the notebook directly!
        name="model",
        input_example=pd.DataFrame({"question": ["What is in the annual report?"]}),
        extra_pip_requirements=[
            "langgraph",
            "langchain",
            "databricks_langchain",
            "databricks-vectorsearch"
        ]
    )
    
    # Log parameters
    mlflow.log_params({
        "vector_index": full_index_name,
        "vector_endpoint": VECTOR_SEARCH_ENDPOINT,
        "llm_endpoint": LLM_ENDPOINT,
        "top_k_results": TOP_K_RESULTS
    })
    
    model_run_id = run.info.run_id
    model_uri = f"runs:/{model_run_id}/model"
    
    print(f"✓ Model logged successfully with MLflow Model As Code!")
    print(f"Run ID: {model_run_id}")
    print(f"Model URI: {model_uri}")


## 4. Create Evaluation Dataset


In [ ]:
# Set experiment before dataset operations (required for MLflow 3.6+)
mlflow.set_experiment(EVAL_EXPERIMENT_NAME)

# implement a basic get or create command
def get_or_create_dataset(uc_table_name: str, **kwargs):
    """Get existing dataset or create new one if it doesn't exist."""
    try:
        # Try to get existing dataset by UC table name
        dataset = get_dataset(name=uc_table_name)
        print(f"Found existing dataset: {uc_table_name}")
        return dataset
    except Exception as e:
        # Dataset doesn't exist, create it
        print(f"Dataset not found, creating: {uc_table_name}")
        dataset = create_dataset(uc_table_name=uc_table_name, **kwargs)
        return dataset

# Define test cases as list of dicts (MLflow 3.6+ format)
test_cases = [
    {
        "inputs": {"question": "What are the key financial highlights from the annual report?"},
        "expectations": {"expected_response": "The answer should mention specific financial metrics from the annual report."}
    },
    {
        "inputs": {"question": "What are the main risks mentioned in the report?"},
        "expectations": {"expected_response": "The answer should identify and describe key business risks."}
    },
    {
        "inputs": {"question": "What is the company's growth strategy?"},
        "expectations": {"expected_response": "The answer should outline the company's strategic growth plans."}
    },
    {
        "inputs": {"question": "What are the major revenue sources?"},
        "expectations": {"expected_response": "The answer should list major revenue streams and their contributions."}
    },
    {
        "inputs": {"question": "What sustainability initiatives are mentioned?"},
        "expectations": {"expected_response": "The answer should describe environmental and social initiatives."}
    },
    {
        "inputs": {"question": "Who are the key executives mentioned?"},
        "expectations": {"expected_response": "The answer should name key leadership team members."}
    },
    {
        "inputs": {"question": "What are the future outlook and projections?"},
        "expectations": {"expected_response": "The answer should discuss future plans and financial projections."}
    },
    {
        "inputs": {"question": "What challenges does the company face?"},
        "expectations": {"expected_response": "The answer should identify main challenges and obstacles."}
    }
]

# Create or get evaluation dataset in Unity Catalog
eval_dataset = get_or_create_dataset(
    uc_table_name=EVAL_DATASET_UC_TABLE,
)

# Add test cases to the dataset (merge_records handles duplicates)
eval_dataset.merge_records(test_cases)

print(f"✓ Evaluation dataset created/updated in Unity Catalog")
print(f"  Table: {EVAL_DATASET_UC_TABLE}")
print(f"  Test cases: {len(test_cases)}")
print("\nSample questions:")
for i, tc in enumerate(test_cases[:3], 1):
    print(f"{i}. {tc['inputs']['question']}")


## 5. Define Custom Scorers


In [ ]:
from typing import Literal

# Custom code-based scorer for response quality metrics
@scorer
def response_quality(inputs: dict, outputs: str) -> dict:
    """Calculate basic quality metrics for the response"""
    response = str(outputs) if outputs else ""
    question = inputs.get('question', '') if isinstance(inputs, dict) else str(inputs)
    
    return {
        "response_length": len(response),
        "word_count": len(response.split()),
        "has_answer": len(response) > 50,
        "mentions_context": "context" in response.lower() or "document" in response.lower()
    }

# Custom LLM-as-judge scorer for relevance using make_judge
relevance_judge = make_judge(
    name="relevance",
    instructions=(
        "Evaluate how relevant and accurate the answer is to the question.\n\n"
        "Question: {{ inputs/question }}\n"
        "Answer: {{ outputs }}\n"
        "Expected Answer Context: {{ expectations/expected_response }}\n\n"
        "Rate the relevance as:\n"
        "- 'highly_relevant': Directly answers the question with accurate information\n"
        "- 'mostly_relevant': Mostly relevant with minor tangential information\n"
        "- 'partially_relevant': Partially relevant but missing key points\n"
        "- 'minimally_relevant': Minimally relevant with significant gaps\n"
        "- 'not_relevant': Not relevant or incorrect\n"
    ),
    feedback_value_type=Literal[
        "highly_relevant",
        "mostly_relevant",
        "partially_relevant",
        "minimally_relevant",
        "not_relevant"
    ],
    model=JUDGE_LLM,
)

# Custom LLM-as-judge scorer for completeness using make_judge
completeness_judge = make_judge(
    name="completeness",
    instructions=(
        "Evaluate how complete and comprehensive the answer is.\n\n"
        "Question: {{ inputs/question }}\n"
        "Answer: {{ outputs }}\n"
        "Expected Answer Context: {{ expectations/expected_response }}\n\n"
        "Rate the completeness as:\n"
        "- 'fully_complete': Fully comprehensive, covers all important aspects\n"
        "- 'mostly_complete': Mostly complete with minor gaps\n"
        "- 'partially_complete': Partially complete, missing some key information\n"
        "- 'incomplete': Incomplete, significant information missing\n"
        "- 'very_incomplete': Very incomplete or lacks substance\n"
    ),
    feedback_value_type=Literal[
        "fully_complete",
        "mostly_complete",
        "partially_complete",
        "incomplete",
        "very_incomplete"
    ],
    model=JUDGE_LLM,
)

# Register all scorers to the experiment (MLflow 3.6+)
# Each scorer gets a unique name for tracking and retrieval

# Built-in scorers with registration
safety_scorer = Safety().register(name="safety_judge")
relevance_to_query_scorer = RelevanceToQuery().register(name="relevance_to_query_judge")

# Register custom code-based scorer
response_quality_scorer = response_quality.register(name="response_quality_metrics")

# Register LLM-as-judge scorers
relevance_judge_registered = relevance_judge.register(name="custom_relevance_judge")
completeness_judge_registered = completeness_judge.register(name="custom_completeness_judge")

# Combine all registered scorers for evaluation
all_scorers = [
    safety_scorer,
    relevance_to_query_scorer,
    response_quality_scorer,
    relevance_judge_registered,
    completeness_judge_registered
]

print("✓ Scorers defined and registered to experiment:")
print("  Built-in (registered):")
print("    - safety_judge (Safety)")
print("    - relevance_to_query_judge (RelevanceToQuery)")
print("  Code-based (registered):")
print("    - response_quality_metrics")
print("  LLM Judges (registered):")
print("    - custom_relevance_judge")
print("    - custom_completeness_judge")

# List all registered scorers in the experiment
print("\n✓ Registered scorers in experiment:")
for s in list_scorers():
    print(f"  - {s.name}")


## 6. Run MLflow GenAI Evaluation


In [ ]:
# View aggregate metrics from the evaluation
print("=" * 60)
print("AGGREGATE METRICS")
print("=" * 60)

metrics_df = pd.DataFrame([results.metrics]).T
metrics_df.columns = ["Value"]
metrics_df.index.name = "Metric"
display(metrics_df)

In [ ]:
# View per-row evaluation results
print("=" * 60)
print("PER-ROW EVALUATION RESULTS")
print("=" * 60)

eval_results_df = results.eval_results

# Show key columns for readability
display_cols = [col for col in eval_results_df.columns if col not in ['trace']]
display(eval_results_df[display_cols])

## 6. Run MLflow GenAI Evaluation


In [ ]:
# Analyze LLM judge scores distribution
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Relevance score distribution
if 'relevance' in eval_results_df.columns:
    relevance_counts = eval_results_df['relevance'].value_counts()
    axes[0].bar(relevance_counts.index, relevance_counts.values, color='steelblue')
    axes[0].set_title('Relevance Score Distribution', fontsize=12)
    axes[0].set_xlabel('Relevance Rating')
    axes[0].set_ylabel('Count')
    axes[0].tick_params(axis='x', rotation=45)

# Completeness score distribution
if 'completeness' in eval_results_df.columns:
    completeness_counts = eval_results_df['completeness'].value_counts()
    axes[1].bar(completeness_counts.index, completeness_counts.values, color='darkorange')
    axes[1].set_title('Completeness Score Distribution', fontsize=12)
    axes[1].set_xlabel('Completeness Rating')
    axes[1].set_ylabel('Count')
    axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze response quality metrics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Response length distribution
if 'response_quality/response_length' in eval_results_df.columns:
    lengths = eval_results_df['response_quality/response_length']
    axes[0].hist(lengths, bins=10, color='seagreen', edgecolor='black')
    axes[0].axvline(lengths.mean(), color='red', linestyle='--', label=f'Mean: {lengths.mean():.0f}')
    axes[0].set_title('Response Length Distribution', fontsize=12)
    axes[0].set_xlabel('Character Count')
    axes[0].set_ylabel('Frequency')
    axes[0].legend()

# Word count distribution
if 'response_quality/word_count' in eval_results_df.columns:
    word_counts = eval_results_df['response_quality/word_count']
    axes[1].hist(word_counts, bins=10, color='mediumpurple', edgecolor='black')
    axes[1].axvline(word_counts.mean(), color='red', linestyle='--', label=f'Mean: {word_counts.mean():.0f}')
    axes[1].set_title('Word Count Distribution', fontsize=12)
    axes[1].set_xlabel('Word Count')
    axes[1].set_ylabel('Frequency')
    axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Generate summary insights
print("=" * 60)
print("EVALUATION INSIGHTS SUMMARY")
print("=" * 60)

# Safety analysis
if 'Safety/safe' in eval_results_df.columns:
    safe_pct = (eval_results_df['Safety/safe'].sum() / len(eval_results_df)) * 100
    print(f"\n🛡️  Safety: {safe_pct:.1f}% of responses are safe")

# Relevance analysis
if 'RelevanceToQuery/relevant_to_query' in eval_results_df.columns:
    relevant_pct = (eval_results_df['RelevanceToQuery/relevant_to_query'].sum() / len(eval_results_df)) * 100
    print(f"🎯 Query Relevance: {relevant_pct:.1f}% of responses are relevant")

# Custom relevance judge analysis
if 'relevance' in eval_results_df.columns:
    high_relevance = eval_results_df['relevance'].isin(['highly_relevant', 'mostly_relevant']).sum()
    high_rel_pct = (high_relevance / len(eval_results_df)) * 100
    print(f"📊 High Relevance (Judge): {high_rel_pct:.1f}% scored highly/mostly relevant")

# Completeness analysis
if 'completeness' in eval_results_df.columns:
    high_complete = eval_results_df['completeness'].isin(['fully_complete', 'mostly_complete']).sum()
    high_comp_pct = (high_complete / len(eval_results_df)) * 100
    print(f"✅ High Completeness (Judge): {high_comp_pct:.1f}% scored fully/mostly complete")

# Response quality metrics
if 'response_quality/word_count' in eval_results_df.columns:
    avg_words = eval_results_df['response_quality/word_count'].mean()
    print(f"\n📝 Average Response Length: {avg_words:.0f} words")

if 'response_quality/has_answer' in eval_results_df.columns:
    has_answer_pct = (eval_results_df['response_quality/has_answer'].sum() / len(eval_results_df)) * 100
    print(f"💬 Substantive Answers: {has_answer_pct:.1f}% have substantive responses")

print("\n" + "=" * 60)

In [ ]:
# Identify areas for improvement
print("=" * 60)
print("AREAS FOR IMPROVEMENT")
print("=" * 60)

# Find low-performing questions based on relevance and completeness judges
low_performers = []

if 'relevance' in eval_results_df.columns and 'completeness' in eval_results_df.columns:
    for idx, row in eval_results_df.iterrows():
        issues = []
        if row.get('relevance') in ['partially_relevant', 'minimally_relevant', 'not_relevant']:
            issues.append(f"Low relevance: {row['relevance']}")
        if row.get('completeness') in ['partially_complete', 'incomplete', 'very_incomplete']:
            issues.append(f"Low completeness: {row['completeness']}")
        
        if issues:
            question = row['inputs'].get('question', 'Unknown') if isinstance(row['inputs'], dict) else str(row['inputs'])
            low_performers.append({
                'question': question[:80] + '...' if len(question) > 80 else question,
                'issues': ', '.join(issues)
            })

if low_performers:
    print("\n⚠️  Questions that need attention:\n")
    for i, item in enumerate(low_performers, 1):
        print(f"{i}. {item['question']}")
        print(f"   Issues: {item['issues']}\n")
else:
    print("\n✅ All questions performed well across relevance and completeness metrics!")

# Overall recommendation
print("\n" + "-" * 60)
print("RECOMMENDATIONS:")
print("-" * 60)

recommendations = []
if 'relevance' in eval_results_df.columns:
    low_rel = eval_results_df['relevance'].isin(['partially_relevant', 'minimally_relevant', 'not_relevant']).sum()
    if low_rel > len(eval_results_df) * 0.3:
        recommendations.append("• Consider improving retrieval quality - many responses lack relevance")

if 'completeness' in eval_results_df.columns:
    low_comp = eval_results_df['completeness'].isin(['partially_complete', 'incomplete', 'very_incomplete']).sum()
    if low_comp > len(eval_results_df) * 0.3:
        recommendations.append("• Responses may need more context - consider increasing TOP_K_RESULTS")

if 'response_quality/word_count' in eval_results_df.columns:
    avg_words = eval_results_df['response_quality/word_count'].mean()
    if avg_words < 50:
        recommendations.append("• Responses are quite short - consider adjusting prompts for more detailed answers")

if not recommendations:
    recommendations.append("• Model is performing well overall!")
    recommendations.append("• Consider adding more diverse test cases to validate robustness")

for rec in recommendations:
    print(rec)

In [ ]:
# Link to MLflow UI for detailed exploration
print("=" * 60)
print("EXPLORE IN MLFLOW UI")
print("=" * 60)
print(f"\nEvaluation Run ID: {eval_run_id}")
print(f"Dataset: {EVAL_DATASET_UC_TABLE}")
print(f"\nView detailed results, traces, and artifacts in the MLflow Experiment UI:")
print(f"Experiment: {EVAL_EXPERIMENT_NAME}")
print("\nThe MLflow UI provides:")
print("• Interactive trace viewer for each evaluation")
print("• Scorer feedback and rationale details")
print("• Artifact downloads and comparison tools")
print("• Registered scorers and datasets in Unity Catalog")

## 7. Production Monitoring Setup (Optional)


In [ ]:
# Start production monitoring with scorer sampling (MLflow 3.6+)
# This enables continuous evaluation of production traces

# Start safety scorer for production monitoring
# Sample rate determines what percentage of traces are evaluated
safety_monitor = safety_scorer.start(
    sampling_config=ScorerSamplingConfig(sample_rate=PRODUCTION_SAMPLE_RATE)
)

# Start relevance scorer for production monitoring
relevance_monitor = relevance_to_query_scorer.start(
    sampling_config=ScorerSamplingConfig(sample_rate=PRODUCTION_SAMPLE_RATE)
)

print("✓ Production monitoring started:")
print(f"  - safety_judge: {PRODUCTION_SAMPLE_RATE*100:.0f}% sample rate")
print(f"  - relevance_to_query_judge: {PRODUCTION_SAMPLE_RATE*100:.0f}% sample rate")
print("\nMonitored scorers will automatically evaluate incoming traces.")
print("View results in MLflow Experiment UI under 'Traces' tab.")


In [ ]:
# Stop production monitoring when done (cleanup)
# Uncomment to stop the monitors:
# safety_monitor.stop()
# relevance_monitor.stop()
# print("✓ Production monitoring stopped")


In [ ]:
# Run evaluation using Unity Catalog dataset
print("Running evaluation...\n")
print("This may take a few minutes...\n")

with mlflow.start_run(run_name="rag_agent_evaluation") as eval_run:
    
    # Load the model for prediction
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    
    # Define prediction function
    def predict_fn(question: str) -> str:
        """
        Prediction function that takes a question string and returns a prediction.
        MLflow will call this with: predict_fn(question='What is...')
        """
        model_input = pd.DataFrame({"question": [question]})
        result = loaded_model.predict(model_input)
        return result if isinstance(result, str) else result[0]
    
    print(f"Using evaluation dataset: {EVAL_DATASET_UC_TABLE}")
    print(f"Scorers to run: {len(all_scorers)}")
    
    # Evaluate using Unity Catalog dataset and registered scorers
    results = evaluate(
        data=eval_dataset,  # Use UC dataset instead of inline DataFrame
        predict_fn=predict_fn,
        scorers=all_scorers  # Registered scorers
    )
    
    eval_run_id = eval_run.info.run_id
    
    print(f"\n✓ Evaluation completed!")
    print(f"Evaluation Run ID: {eval_run_id}")
    print(f"Dataset: {EVAL_DATASET_UC_TABLE}")
